<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Hometasks/Base/ML_AI24_HT7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание 7: Fraud Detection Competition**

Добро пожаловать на финальное домашнее задание курса! Домашнее задание направлено на систематизацию всех знаний, полученных в процессе учебы.

В нём Вы потренируетесь применять навыки построения пайплайнов машинного обучения, приобретенные в курсе от точки разведочного анализа данных до построения и валидации моделей машинного обучения.

## **Задача**
**Вы будете решать задачу определения фрода:**

https://www.kaggle.com/competitions/fraud-detection-24

**Вам нужно будет:**
- в jupyter notebook провести исследование данных;
- в нём же построить модели и оценить их качество;
- отправить посылку на Kaggle.

Более подробное описание шагов - в ноутбуке ниже.

## **Оценивание и баллы**
- В EDA и во всей работе будут оцениваться полнота и **выводы**;
- При обучении моделей старайтесь обоснованно подходить к их выбору, избегая простого перебора;

**Максимальный балл** - 10 (+ бонусы за Kaggle, см. ниже).


Мягкий дедлайн (окончание соревнования на Kaggle): **15 марта 23:59**


# **Базовое решение и пример формирования файла под submission**


In [1]:
# # Создаем директорию data, если она не существует
# !mkdir -p data

# # Скачиваем файлы в директорию data
# !wget --no-check-certificate 'https://www.dropbox.com/s/5iuef7c9ljj84t6/train_transaction.csv?dl=0' -O data/train_transaction.csv
# !wget --no-check-certificate 'https://www.dropbox.com/s/cmy01z5fw7ohlmd/train_identity.csv?dl=0' -O data/train_identity.csv
# !wget --no-check-certificate 'https://www.dropbox.com/s/7thqkuxnwsa7njj/test_transaction.csv?dl=0' -O data/test_transaction.csv
# !wget --no-check-certificate 'https://www.dropbox.com/s/b40nvbb9e2usd5w/test_identity.csv?dl=0' -O data/test_identity.csv
# !wget --no-check-certificate 'https://www.dropbox.com/s/arkyoz0bel8z4d2/sample_submission.csv?dl=0' -O data/sample_submission.csv

In [2]:
# # Скачиваем файлы с Kaggle
# !pip install kaggle -q

# # Убедимся, что директория для API ключа Kaggle существует
# !mkdir -p ~/.kaggle

# # Создаем файл с API ключом Kaggle (замените на свой ключ)
# # Примечание: в реальном использовании лучше загрузить ключ из безопасного хранилища
# # !echo '{"username":"your_username","key":"your_api_key"}' > ~/.kaggle/kaggle.json
# # !chmod 600 ~/.kaggle/kaggle.json

# # Скачиваем файлы соревнования
# !kaggle competitions download -c fraud-detection-24 -p data
# !unzip -o data/fraud-detection-24.zip -d data


In [3]:
!pip install catboost -q

In [4]:
from catboost import CatBoostClassifier
import pandas as pd
import os
from tqdm import tqdm

INPUT_DIR = 'data'

train_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'train_transaction.csv'))
train_identity = pd.read_csv(os.path.join(INPUT_DIR, 'train_identity.csv'))
test_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'test_transaction.csv'))
test_identity = pd.read_csv(os.path.join(INPUT_DIR, 'test_identity.csv'))
sample_submission = pd.read_csv(os.path.join(INPUT_DIR, 'sample_submission.csv'))

df_train = train_transaction.merge(train_identity, how='left', on='TransactionID')

df_test = test_transaction.merge(test_identity, how='left', on='TransactionID')

In [5]:
df_train.drop(['TransactionID', 'TransactionDT'], axis=1, inplace=True)
df_test.drop(['TransactionID', 'TransactionDT'], axis=1, inplace=True)
df_train.shape, df_test.shape

((417559, 432), (172981, 431))

In [6]:
object_cols = []
for idx, col in tqdm(enumerate(df_train.columns.drop('isFraud'))):
    if df_train[col].dtype == 'O':
      object_cols.append(idx)

431it [00:00, 83030.73it/s]


In [7]:
X_train = df_train.drop('isFraud', axis=1)
X_train = X_train.fillna('')
y_train = df_train['isFraud'].values

# cb = CatBoostClassifier(iterations=3)
# cb.fit(X_train, y_train, object_cols, )

In [8]:
# predictions = cb.predict_proba(df_test.fillna(''))

In [9]:
# sub = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': predictions[:, 0]})
# sub.head()

In [10]:
# Сохранение submission
# sub.to_csv('submission_baseline.csv', index=False)

In [11]:
# Отправка файла на соревнование
# !kaggle competitions submit -c fraud-detection-24 -f submission_baseline.csv -m "Базовая модель CatBoost"

### **Примечания:**

**1. Оценка качества и Submission File**
- Ответом является число от 0 до 1, метрикой качества - AUC-ROC.
- Структура Submission File:
 - для каждого значения *TransactionID* в тестовых данных вы должны предсказать **вероятность** для столбца *isFraud*.
 - в файле у вас должно быть две колонки: `TransactionID` и`isFraud`  **для каждой транзакции в датасете**.

**2. Объем данных**

Поскольку набор данных объемный, могут быть проблемы с переполнением памяти в Collab. Для решения проблемы можете использовать функцию из [этого ноутбука](https://colab.research.google.com/drive/18u75eyFGEoyeWJ_MbsLkcPa6gv2tNI8G#scrollTo=V2L1Nl5CTMMl), разобравшись, что она делает с данными.

# **Задание 1 (2 балла)**

Проведите EDA (разведочный анализ данных):
* проанализируйте признаки, их особенности и связь с целевой переменной
* проанализируйте целевую переменную
* оцените степень попарной взаимосвязи признаков, а также связи признаков и целевой переменной
* по результатам исследований сделайте необходимую обработку данных (удаление дублей, работа с пропусками, с категориальными столбцами (если они есть), работа с аномалиями, другие преобразования признаков)

Важно, что EDA всегда сопровождается выводами - не забудьте об этом.

## Анализ признаков

In [12]:
# ваш EDA здесь
print("\nИнформация о тренировочном наборе:")
df_train.info(verbose=False)


Информация о тренировочном наборе:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417559 entries, 0 to 417558
Columns: 432 entries, isFraud to DeviceInfo
dtypes: float64(399), int64(2), object(31)
memory usage: 1.3+ GB


#вывод
- В обучающей выборке 432 признака, включая целевую `isFraud`.
- dtypes: float64(399), int64(2), object(31)
- датасет занимает 1.3ГБ памяти, что много. Возможно типы данных неэффективно выбораны.

### Оптимизируем типы данных

In [13]:
import numpy as np

In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm


def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Использование памяти до оптимизации: {:.2f} MB'.format(start_mem))
    for col in tqdm(df.columns):
        if df[col].dtype not in ['object', 'category']: 

            is_NAN_or_INF = np.isfinite(df[col]).any() or df[col].isna().any()

            if np.all(np.abs(df[col].dropna() - df[col].dropna().round()) < 1e-15):
                rounded = df[col].dropna().round()
                rmin = rounded.min()
                rmax = rounded.max()

                if rmin >= 0:
                    if rmax < np.iinfo(np.uint8).max:
                        dtype_nullable = 'UInt8'
                        dtype_regular = np.uint8
                    elif rmax < np.iinfo(np.uint16).max:
                        dtype_nullable = 'UInt16'
                        dtype_regular = np.uint16
                    elif rmax < np.iinfo(np.uint32).max:
                        dtype_nullable = 'UInt32'
                        dtype_regular = np.uint32
                    else:
                        dtype_nullable = 'UInt64'
                        dtype_regular = np.uint64
                else:
                    if rmin > np.iinfo(np.int8).min and rmax < np.iinfo(np.int8).max:
                        dtype_nullable = 'Int8' 
                        dtype_regular = np.int8
                    elif rmin > np.iinfo(np.int16).min and rmax < np.iinfo(np.int16).max:
                        dtype_nullable = 'Int16'
                        dtype_regular = np.int16
                    elif rmin > np.iinfo(np.int32).min and rmax < np.iinfo(np.int32).max:
                        dtype_nullable = 'Int32'
                        dtype_regular = np.int32
                    elif rmin > np.iinfo(np.int64).min and rmax < np.iinfo(np.int64).max:
                        dtype_nullable = 'Int64'
                        dtype_regular = np.int64
                # Применяем выбранный тип данных
                try:
                    df[col] = df[col].astype(dtype_nullable) if is_NAN_or_INF else rounded.astype(dtype_regular)
                except:
                    print(f"Ошибка на {col}: {df[col].dtype} -> {dtype_nullable=}, {dtype_regular=}, {is_NAN_or_INF=}, {rmin=}, {rmax=}")
                    df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float32)
        else:  # тип object пытаемся преобразовать в category
            try:
                df[col] = df[col].astype('category')
            except:
                print(f"Ошибка на {col}: {df[col].dtype} -> 'category'")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Использование памяти после оптимизации: {:.2f} MB'.format(end_mem))
    print('Уменьшение памяти на {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df


In [15]:
df_train = reduce_mem_usage(df_train)

Использование памяти до оптимизации: 1376.23 MB


100%|██████████| 432/432 [00:12<00:00, 33.55it/s]

Использование памяти после оптимизации: 416.20 MB
Уменьшение памяти на 69.8%


In [16]:
df_test = reduce_mem_usage(df_test)

Использование памяти до оптимизации: 568.81 MB


100%|██████████| 431/431 [00:05<00:00, 75.73it/s] 

Использование памяти после оптимизации: 174.10 MB
Уменьшение памяти на 69.4%


#вывод
| Занято памяти | До оптимизации | После оптимизации | Уменьшение |
|---------------|----------------|-------------------|------------|
| df_train      | 1376.23 MB     | 416.2 MB          | 69.8%      |
| df_test       |  568.81 MB     | 174.1 MB          | 69.4%      |

### Пропуски

In [18]:
missing_values = df_train.isnull().sum()

In [ ]:
missing_values[missing_values > 0].count()

# **Задание 2 (3 балла)**

Обучите несколько ML-моделей для решения поставленной задачи.
Оцените их качество двумя способами:

1) на кросс-валидации

2) на лидерборде

Подберите число фолдов на кросс-валидации так, чтобы метрики, которые вы видите, были максимально близки на кросс-валидации и на лидерборде.

По результатам экспериментов постройте таблицу:
* в каждой строке таблицы - результаты одной модели
* по столбцам: качество на кросс-валидации, качество на лидерборде, модель с гиперпараметрами
Полученную таблицу вставьте картинкой прямо в ноутбук после ячеек с кодом. Сделайте текстовые выводы.

In [ ]:
# ваши модели здесь

# **Задание 3 (2 балла)**

Попробуйте подойти к задаче как к поиску аномалий.

1) Поищите аномалии (фрод) различными рассмотренными в курсе методами и сделайте прогноз на тестовых данных.

Результатом также будет таблица:
* по строкам - методы поиска аномалий
* по столбцам - качество вашего решения на leaderboard

2) Попробуйте встроить поиск аномалий и их удаление в ML-пайплайн: найдите аномалии и что-нибудь с ними сделайте до обучения моделей (можно удалить их, а можно использовать в качестве дополнительных признаков - попробуйте разные стратегии). Результат проверьте на кросс-валидации и на лидерборде, сделайте выводы.

In [ ]:
# ваша работа с аномалиями здесь

# **Задание 4 (1 балл)**

Сделайте кластеризацию различными способами. Результаты кластеризации используйте для улучшения ML-решений:

1) Номера кластеров закодируйте (OHE или target-encoding) и добавьте как новые признаки

2) При использовании DBSCAN / HDBSCAN предсказанный шум можно трактовать как найденную аномалию и также добавить ее как новый признак

Проведите различные эксперименты. Проверьте как эти подходы влияют на качество прогнозов по кросс-валидации и на лидерборде, сделайте выводы.

In [ ]:
# ваши эксперименты с кластеризацией здесь

## **Задание 5 (1 балл)**

Примените какой-нибудь (один любой) AutoML фреймворк для решения поставленной задачи.

Отправьте AutoML-прогноз на kaggle и посмотрите на качество модели. Сделайте текстовые выводы.

In [ ]:
# ваш AutoML здесь

# **Задание 6 (1 балл)**

Весь курс мы работали в Google Colab. Но всегда должны быть запасные варианты, где Вы будете обучать модели.

Среди вариантов есть:
* ваша локальная машина
* kaggle notebooks
* yandex cloud
и другие.

Кроме привычного Google Colab выберите из списка выше один любой альтернативный вариант и проведите эксперимент:

* Прогоните ваш лучший по качеству по результатам заданий 2-4 ML-пайплайн заново в Google Colab и с помощью библиотек (например, при помощи библиотеки time) замерьте время обучения и отдельно время инференса на тестовых данных

* Прогоните этот пайплайн на выбранном альтернативном сервисе/локальной машине и также замерьте время обучения и инференса.

Текстом напишите выводы: опишите, какое альтернативное место для обучения моделей Вы использовали? Прикрепите прямо в ноутбук скриншот с экраном кода в альтернативном сервисе/на локальной машине. Также в виде таблицы приведите сравнение времени обучения и инференса в колабе и в альтернативном месте. Сделайте выводы.

In [ ]:
# ваши эксперименты здесь

# **Бонус: за Kaggle и стремление к хорошим скорам (2 балла)**

В этом домашнем задании Ваша цель - не просто выполнить шаги выше, но и построить максимально хорошую по качеству модель.

**К 10 вы можете получить до двух дополнительных баллов:**

* За попадание в топ-20% на private leaderboard — +1 дополнительный балл к оценке
* За попадание в топ-5 мест на private leaderboard — + еще один дополнительный балл к оценке (то есть суммарно 2 дополнительных балла)

**ВАЖНО!!!**

Эти баллы ставятся до мягкого дедлайна по соревнованию. После мягкого дедлайна лидерборд не обновляется, и дополнительные баллы не ставятся.

Успехов!

In [ ]:
# не забудьте прикрепить скриншоты лидерборда, пожалуйста